## 1. 환경 설정

`(1) LangSmith 설정 확인`
- .env 파일에 아래 내용을 반영
    - LANGCHAIN_TRACING_V2=true  
    - LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"  
    - LANGCHAIN_API_KEY="인증키를 입력하세요"  
    - LANGCHAIN_PROJECT="프로젝트명"  

`(2) 기본 라이브러리`

In [1]:
import os
from glob import glob

from pprint import pprint
import json

import numpy as np
import pandas as pd

`(3) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
print("langsmith 추척 여부: ", os.getenv('LANGCHAIN_TRACING_V2'))

## 2. Load Data

`(1) Raw Documents`

In [ ]:
# 문서를 로드
from langchain_core.documents import Document
import json

final_docs = []

with open('./data/final_docs_ver2.jsonl', 'rb') as f:
    for line in f:
        item = json.loads(line)
        doc = Document(page_content=item['page_content'], metadata=item['metadata'])
        final_docs.append(doc)

print(len(final_docs))

`(2) Test Data`

In [ ]:
# Test 데이터셋에 대한 QA 생성 결과를 리뷰한 후 다시 로드
import pandas as pd

df_qa_test = pd.read_excel("./data/qa_test_revised.xlsx")

df_qa_test.head()

`(3) 검색도구 정의`

-  BM25 검색기

In [ ]:
# BM25 검색기를 사용하기 위한 준비
from krag.tokenizers import KiwiTokenizer
from krag.retrievers import KiWiBM25RetrieverWithScore

kiwi_tokenizer = KiwiTokenizer(model_type='knlm', typos='basic')

bm25_db = KiWiBM25RetrieverWithScore(
        documents=final_docs, 
        kiwi_tokenizer=kiwi_tokenizer, 
        k=2, 
        threshold=0.0,
    )

# BM25 검색기를 사용하여 문서 검색
query = "테슬라의 회장은 누구인가요?"
retrieved_docs = bm25_db.invoke(query, 2)

# 검색 결과 출력 
print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

- Vector Store 로드

In [ ]:
# 벡터스토어 로드
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

chroma_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="hf_bge_m3",
    persist_directory="./chroma_db",
)

chroma_k = chroma_db.as_retriever(
    search_kwargs={'k': 2},
)

# 벡터스토어를 사용하여 문서 검색
query = "테슬라의 회장은 누구인가요?"

retrieved_docs = chroma_k.invoke(query)

# 검색 결과 출력
print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

- Emsemble Hybrid Search 활용

In [ ]:
from langchain.retrievers import EnsembleRetriever


# 검색기 초기화 
def create_hybrid_retriever(bm25_db, vector_db, k: int = 4):

    bm25_db.k = k
    chroma_k = vector_db.as_retriever(search_kwargs={'k': k})

    retriever = EnsembleRetriever(
        retrievers=[bm25_db, chroma_k],
        weights=[0.5, 0.5],
    )

    return retriever

hybrid_retriever = create_hybrid_retriever(bm25_db, chroma_db, k=4)

query = "테슬라의 회장은 누구인가요?"
retrieved_docs = hybrid_retriever.invoke(query)

# 검색 결과 출력
print(f"쿼리: {query}")
print("검색 결과:")

for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

- Cross-Encoder 알고리즘에 기반하여 재정렬 (Re-rank)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
re_ranker = CrossEncoderReranker(model=model, top_n=3)

cross_encoder_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker, 
    base_retriever=hybrid_retriever,
)

question = "테슬라 회장은 누구인가요?"

retrieved_docs = cross_encoder_reranker_retriever.invoke(question)

print(f"쿼리: {question}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

## 3. LLM 유형 및 답변 생성

### 3-1 RAG Chain

In [18]:
# 각 쿼리에 대한 검색 결과를 한꺼번에 Context로 전달해서 답변을 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def create_rag_chain(retriever, llm):

    template = """Answer the following question based on this context. If the context is not relevant to the question, just answer with '답변에 필요한 근거를 찾지 못했습니다.'

    [Context]
    {context}

    [Question]
    {question}

    [Answer]
    """

    prompt = ChatPromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n".join([f"{doc.page_content}" for doc in docs])

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()} 
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

In [ ]:
# RAG 체인 생성 및 테스트
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

openai_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = openai_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)


### 3-2 주요 모델 공급자

`(1) Anthropic Claude API`

https://www.anthropic.com/api

In [ ]:
# ChatAnthropic - LLM 모델 
from langchain_anthropic import ChatAnthropic

# 모델 로드 
llm = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0,
    max_tokens=200, 
)

# RAG 체인 생성 및 테스트
anthropic_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = anthropic_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)

`(2) Google Gemini API`

https://ai.google.dev/

In [ ]:
# ChatGoogleGenerativeAI - LLM 모델 
from langchain_google_genai import ChatGoogleGenerativeAI

# 모델 로드 
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# RAG 체인 생성 및 테스트
google_genai_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = google_genai_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)


`(3) Ollama - 오픈소스 LLM`

https://ollama.com/

In [ ]:
# ChatOllama - LLM 모델 
from langchain_ollama import ChatOllama

# 모델 로드 
llm = ChatOllama(
    model = "llama3.1",
    temperature = 0.8,
    num_predict = 100,
)

# RAG 체인 생성 및 테스트
ollama_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = ollama_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)

`(4) Groq API - 오픈소스 LLM`

https://groq.com/

In [ ]:
# ChatGroq - LLM 모델 
from langchain_groq import ChatGroq

# 모델 로드 
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.0,
    max_tokens=100,
)

# RAG 체인 생성 및 테스트
groq_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = groq_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)

## 4. RAG 답변 평가

### 4-1. Metric Evaluation

`(1) Embedding Distance`
- 임베딩 거리를 사용하여 예측 문자열과 참조 레이블 문자열 간의 의미적 유사성을 측정
- 계산된 거리 점수가 낮을수록 두 문자열의 의미가 더 유사함을 나타내며, 이 방법은 단순 문자열 비교보다 더 풍부한 의미적 평가가 가능

In [ ]:
# LangChain EmbeddingDistanceEvalChain 활용 

from langchain.evaluation import load_evaluator, EvaluatorType, EmbeddingDistance
from langchain_openai import ChatOpenAI

# Evaluator 초기화
embedding_evaluator = load_evaluator(
    evaluator=EvaluatorType.EMBEDDING_DISTANCE,      # 임베딩 거리를 기반으로 평가
    distance_metric=EmbeddingDistance.COSINE,        # 코사인 유사도 사용
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0), # OpenAI LLM 사용
    )


# 의미가 다른 문장 비교
result1 = embedding_evaluator.evaluate_strings(prediction="나는 학교에 갈 것이다", reference="나는 집에 있을 것이다")
print("의미가 다른 문장 비교 결과:", result1)

# 의미가 비슷한 문장 비교
result2 = embedding_evaluator.evaluate_strings(prediction="나는 학교에 갈 것이다", reference="나는 학교로 향할 것이다")
print("의미가 비슷한 문장 비교 결과:", result2)

In [ ]:
# test 데이터셋에 대한 평가
df_qa_test.head()

In [ ]:
# 첫번째 샘플에 대해 평가 - ground truth와 비교

question = df_qa_test.iloc[0]['question']
ground_truth = df_qa_test.iloc[0]['answer']

print("Question:", question)
print("Ground Truth:", ground_truth)

# OpenAI LLM을 사용하여 예측 생성
openai_prediction = openai_rag_chain.invoke(question)
print("Prediction:", openai_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=openai_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

In [ ]:
### 다른 모델들에 대해 평가
# Anthropiic LLM을 사용하여 예측 생성
anthropic_prediction = anthropic_rag_chain.invoke(question)
print("Prediction:", anthropic_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=anthropic_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

In [ ]:
# Google Generative AI LLM을 사용하여 예측 생성
google_genai_prediction = google_genai_rag_chain.invoke(question)
print("Prediction:", google_genai_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=google_genai_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

In [ ]:
# Ollama LLM을 사용하여 예측 생성
ollama_prediction = ollama_rag_chain.invoke(question)
print("Prediction:", ollama_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=ollama_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

In [ ]:
# Groq LLM을 사용하여 예측 생성
groq_prediction = groq_rag_chain.invoke(question)
print("Prediction:", groq_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=groq_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

In [ ]:
# 모든 모델에 대해 평가 결과를 비교

def embedding_evaluate_all_models(question, ground_truth):
    results = {}
    for model_name, rag_chain in {
        "OpenAI": openai_rag_chain,
        "Anthropic": anthropic_rag_chain,
        "Google": google_genai_rag_chain,
        "Ollama": ollama_rag_chain,
        "Groq": groq_rag_chain,
    }.items():
        prediction = rag_chain.invoke(question)
        distance_score = embedding_evaluator.evaluate_strings(prediction=prediction, reference=ground_truth)
        results[model_name] = f"{distance_score['score']:.3f}"
    return results

question = df_qa_test.iloc[0]['question']
ground_truth = df_qa_test.iloc[0]['answer']

results = embedding_evaluate_all_models(question, ground_truth)
pprint(results)

In [ ]:
# 전체 데이터셋에 대해 A/B 테스트 - 여기서는 3개만 평가 (데이터프레임으로 정리)

def embedding_evaluate_qa_dataset(df_qa_test):

    results = []

    for i in range(3):
        question = df_qa_test.iloc[i]['question']
        ground_truth = df_qa_test.iloc[i]['answer']
        result = embedding_evaluate_all_models(question, ground_truth)
        results.append(result)

    return pd.DataFrame(results)

df_result_embedding = embedding_evaluate_qa_dataset(df_qa_test.iloc[:3])
df_result_embedding 

In [ ]:
# Embedding Distance가 낮을 수록 좋은 결과
df_result_embedding.astype(float).mean().sort_values(ascending=True)

`(2) Cross-Encoder 활용`
- 두 문장을 동시에 인코딩하여 직접적으로 유사성을 평가 (개별 문장을 따로 인코딩하는 일반적인 Bi-Encoder 임베딩 방식과 다름)
- 크로스 인코더는 두 문장의 상호작용을 직접 모델링하므로, 일반적으로 더 정확한 유사성 점수를 제공
- 이 방법은 의미적 유사성을 더 정확하게 측정할 수 있지만, 계산 비용이 더 높다는 점을 유의해야 함

In [37]:
from sentence_transformers import CrossEncoder

def calculate_cross_encoder_similarity(
        query: str, 
        prediction: str, 
        model_name: str = "BAAI/bge-reranker-v2-m3",
        ) -> float:
    """
    주어진 query와 prediction 사이의 의미적 유사성을 계산합니다.

    Args:
    query (str): 기준이 되는 쿼리 문장
    prediction (str): 유사성을 비교할 예측 문장
    model_name (str): 사용할 크로스 인코더 모델 이름 (기본값: "BAAI/bge-reranker-v2-m3")

    Returns:
    float: 두 문장 간의 유사성 점수
    """
    # 크로스 인코더 모델을 불러옵니다.
    cross_encoder_model = CrossEncoder(model_name)

    # 크로스 인코더를 사용하여 유사성 점수를 계산합니다.
    sentence_pairs = [[query, prediction]]
    similarity_scores = cross_encoder_model.predict(sentence_pairs)

    return similarity_scores[0]

In [ ]:
# 첫 번째 샘플에 대해 유사성 점수 계산 - 점수가 높을수록 더 유사함

print("Question:", question)
print("Ground Truth:", ground_truth)
print("Prediction:", openai_prediction)

similarity = calculate_cross_encoder_similarity(ground_truth, openai_prediction)
print(f"유사성 점수: {similarity:.4f}")

In [ ]:
# 모든 모델에 대해 평가 결과를 비교

def cross_encoder_evaluate_all_models(question, ground_truth):
    results = {}
    for model_name, rag_chain in {
        "OpenAI": openai_rag_chain,
        "Anthropic": anthropic_rag_chain,
        "Google": google_genai_rag_chain,
        "Ollama": ollama_rag_chain,
        "Groq": groq_rag_chain,
    }.items():
        prediction = rag_chain.invoke(question)
        print(model_name)
        print(prediction)
        print()
        similarity = calculate_cross_encoder_similarity(ground_truth, prediction)
        results[model_name] = f"{similarity:.3f}"
    return results


# 전체 데이터셋에 대해 A/B 테스트 - 여기서는 3개만 평가 (데이터프레임으로 정리)

def cross_encoder_evaluate_qa_dataset(df_qa_test):

    results = []

    for i in range(3):
        question = df_qa_test.iloc[i]['question']
        ground_truth = df_qa_test.iloc[i]['answer']
        result = cross_encoder_evaluate_all_models(question, ground_truth)
        results.append(result)

    return pd.DataFrame(results)


df_result_cross_encoder = cross_encoder_evaluate_qa_dataset(df_qa_test.iloc[:3])
df_result_cross_encoder

In [ ]:
# Cross Encoder Similarity가 높을 수록 좋은 결과
df_result_cross_encoder.astype(float).mean().sort_values(ascending=False)

`(3) Rouge Metric`
- ROUGE 메트릭은 두 문장의 유사도를 평가하는 데 널리 사용되는 방법으로, 특히 텍스트 요약과 기계 번역 분야에서 유용함
- 단어 중첩을 기반으로 하여 계산이 빠르고 해석이 쉽다는 장점이 있지만, 깊은 의미적 유사성을 포착하는 데는 한계가 있음
- 사용 목적과 컨텍스트에 따라 적절히 선택하거나 다른 메트릭과 조합하여 사용

In [41]:
from korouge_score import rouge_scorer
from typing import List, Dict

def calculate_rouge_similarity(
        query: str, 
        prediction: str, 
        rouge_types: List[str] = ['rouge1', 'rouge2', 'rougeL'],
        ) -> Dict[str, float]:
    
    """
    주어진 쿼리 문장과 예측 문장 사이의 선택된 ROUGE 점수를 계산합니다.

    Args:
    query (str): 기준이 되는 쿼리 문장
    prediction (str): 유사성을 비교할 예측 문장
    rouge_types (List[str]): 계산할 ROUGE 메트릭 리스트 (기본값: ['rouge1', 'rouge2', 'rougeL'])

    Returns:
    Dict[str, float]: 선택된 ROUGE 메트릭의 F1 점수를 포함하는 딕셔너리
    """
    # 입력된 ROUGE 유형의 유효성을 검사합니다.
    valid_rouge_types = set(['rouge1', 'rouge2', 'rougeL'])
    rouge_types = [rt for rt in rouge_types if rt in valid_rouge_types]
    
    if not rouge_types:
        raise ValueError("유효한 ROUGE 유형이 제공되지 않았습니다.")

    # ROUGE scorer 객체를 초기화합니다.
    scorer = rouge_scorer.RougeScorer(rouge_types, use_stemmer=True)

    # ROUGE 점수를 계산합니다.
    scores = scorer.score(query, prediction)

    # 결과를 정리합니다.
    result = {rouge_type: scores[rouge_type].fmeasure for rouge_type in rouge_types}

    return result

In [ ]:
# 첫 번째 샘플에 대해 유사성 점수 계산 - 점수가 높을수록 더 유사함

print("Question:", question)
print("Ground Truth:", ground_truth)
print("Prediction:", openai_prediction)

rouge_scores = calculate_rouge_similarity(ground_truth, openai_prediction, ['rouge1'])
print(f"Rouge 점수: {rouge_scores['rouge1']:.4f}")

In [ ]:
# 모든 모델에 대해 평가 결과를 비교

def rouge_evaluate_all_models(question, ground_truth):
    results = {}
    for model_name, rag_chain in {
        "OpenAI": openai_rag_chain,
        "Anthropic": anthropic_rag_chain,
        "Google": google_genai_rag_chain,
        "Ollama": ollama_rag_chain,
        "Groq": groq_rag_chain,
    }.items():
        prediction = rag_chain.invoke(question)
        rouge_scores = calculate_rouge_similarity(ground_truth, prediction, ['rouge2'])
        results[model_name] = f"{rouge_scores['rouge2']:.3f}"
    return results

# 전체 데이터셋에 대해 A/B 테스트 - 여기서는 3개만 평가 (데이터프레임으로 정리)

def rouge_evaluate_qa_dataset(df_qa_test):
    
        results = []
    
        for i in range(3):
            question = df_qa_test.iloc[i]['question']
            ground_truth = df_qa_test.iloc[i]['answer']
            result = rouge_evaluate_all_models(question, ground_truth)
            results.append(result)
    
        return pd.DataFrame(results)    

df_result_rouge = rouge_evaluate_qa_dataset(df_qa_test.iloc[:3])
df_result_rouge

In [ ]:
# ROUGE 점수가 높을 수록 좋은 결과
df_result_rouge.astype(float).mean().sort_values(ascending=False)

### 4-2. LLM-as-judge
- LLM은 인간과 유사한 판단을 제공할 수 있어, 단순한 단어 중첩 기반 메트릭보다 더 깊은 의미적 관련성을 포착할 수 있음
- ROUGE와 같은 전통적인 메트릭보다 더 정교한 평가를 제공할 수 있지만, LLM의 판단에 의존하기 때문에 완전한 객관성을 보장하기는 어려움
- 또한 API 사용에 따른 비용과 처리 시간이 필요하다는 점을 고려해야 함
- 따라서 여러 평가 방법을 조합하여 사용 필요 (예를 들어, ROUGE 점수로 빠른 초기 스크리닝을 하고, 중요한 케이스에 대해서만 이 LLM 기반 평가를 수행하는 방식을 고려)

`(1) QA Evaluation - 사용자 질문에 대한 정확성, 관련성을 평가 (Y/N, 0/1)`

1. "qa" 평가기: 사용자 질문에 대한 응답의 정확성을 직접적으로 평가

2. "context_qa" 평가기: 더 넓은 맥락을 고려하여 응답의 정확성을 평가

3. "cot_qa" 평가기: CoT(Chain of Thought) 추론을 통해 더 심층적인 평가를 수행


In [ ]:
# 2번째 샘플에 대해 예측 수행
question = df_qa_test.iloc[1]['question']
context = df_qa_test.iloc[1]['context']
ground_truth = df_qa_test.iloc[1]['answer']
groq_prediction = groq_rag_chain.invoke(question)

print("Question:", question)
print("Context:", context)
print("Ground Truth:", ground_truth)
print("Prediction:", groq_prediction)

In [ ]:
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain_openai import ChatOpenAI

# QA Evaluator 초기화
qa_evaluator = load_evaluator(
    evaluator="qa",  
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0), # OpenAI LLM 사용
    )

# 2번째 샘플에 대해 평가 수행
qa_eval_result = qa_evaluator.evaluate_strings(
    input=question,               # 평가에 고려할 내용: 질문
    prediction=groq_prediction,   # 평가 대상: LLM 모델의 예측
    reference=ground_truth,       # 평가 기준: 정답
)

# 결과 출력
qa_eval_result

In [ ]:
# Context QA Evaluator 초기화
context_qa_evaluator = load_evaluator(
    evaluator="context_qa",  
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0), # OpenAI LLM 사용
    )

# 2번째 샘플에 대해 평가 수행
context_qa_eval_result = context_qa_evaluator.evaluate_strings(
    input=question,               # 평가에 고려할 내용: 질문
    prediction=groq_prediction,   # 평가 대상: LLM 모델의 예측
    reference=context,            # 평가 기준: 문맥
)

# 결과 출력
context_qa_eval_result

In [ ]:
# COT QA Evaluator 초기화
cot_qa_evaluator = load_evaluator(
    evaluator="cot_qa",  
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0), # OpenAI LLM 사용
    )

# 2번째 샘플에 대해 평가 수행
cot_qa_eval_result = cot_qa_evaluator.evaluate_strings(
    input=question,               # 평가에 고려할 내용: 질문
    prediction=groq_prediction,   # 평가 대상: LLM 모델의 예측
    reference=context,            # 평가 기준: 문맥
)

# 결과 출력
cot_qa_eval_result

In [ ]:
print(cot_qa_eval_result["reasoning"])

In [ ]:
# COT QA Evaluator 초기화 (사용자 정의 프롬프트 사용)
from langchain_core.prompts.prompt import PromptTemplate

CUSTOM_COT_QA_PROMPT = """
You are an expert evaluating the performance of a RAG (Retrieval-Augmented Generation) system. Your task is to assess the quality of the system's answer based on the given question, retrieved context, and the generated answer. Grade the answer as CORRECT, PARTIALLY CORRECT, or INCORRECT, and provide a detailed explanation. Please describe your evaluation process step by step to clearly show how you reached your conclusion.

Use the following criteria for evaluation:
1. Factual Accuracy: Does the answer align with the information in the retrieved context?
2. Relevance: Does the answer appropriately address the question?
3. Completeness: Does the answer cover all aspects of the question?
4. Conciseness: Does the answer convey the key information without unnecessary details?

Grading Criteria:
- CORRECT: Meets all criteria with no errors
- PARTIALLY CORRECT: Meets some criteria but has minor errors or omissions
- INCORRECT: Contains major factual errors or is irrelevant to the question

Evaluation Format:
QUESTION: [The question content]
CONTEXT: [The retrieved context]
SYSTEM ANSWER: [The answer generated by the RAG system]
EVALUATION:
1. Factual Accuracy: [Assessment and explanation]
2. Relevance: [Assessment and explanation]
3. Completeness: [Assessment and explanation]
4. Conciseness: [Assessment and explanation]
Overall Assessment: [General evaluation summary]
GRADE: [CORRECT / PARTIALLY CORRECT / INCORRECT]

Now, please evaluate the RAG system's answer based on the provided information. Prefer to use in Korean language.

QUESTION: {query}
CONTEXT: {context}
SYSTEM ANSWER: {result}
EVALUATION:
"""

custom_qa_prompt = PromptTemplate.from_template(CUSTOM_COT_QA_PROMPT)

custom_cot_qa_evaluator = load_evaluator(
    evaluator="cot_qa",  
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0), # OpenAI LLM 사용
    prompt=custom_qa_prompt                               # 사용자 지정 프롬프트 사용
    )

# 2번째 샘플에 대해 평가 수행
custom_cot_qa_eval_result = custom_cot_qa_evaluator.evaluate_strings(
    input=question,               # 평가에 고려할 내용: 질문
    prediction=groq_prediction,   # 평가 대상: LLM 모델의 예측
    reference=context,            # 평가 기준: 문맥
)

# 결과 출력
custom_cot_qa_eval_result

In [ ]:
print(custom_cot_qa_eval_result['reasoning'])

`(2) Criteria Evaluation (No lables) - 참조 레이블(ground truth)이 없는 상황에서 모델 출력의 품질을 평가`

1. "criteria" 평가기:
   - 목적: 주어진 기준에 따라 예측이 기준을 만족하는지 평가
   - 출력: 이진 점수 (예: Yes/No 또는 1/0)

2. "score_string" 평가기:
   - 목적: 주어진 기준에 따라 예측의 품질을 수치로 평가
   - 출력: 수치 점수 (기본적으로 1-10 척도)


In [ ]:
from langchain.evaluation import load_evaluator

# 간결성 평가 - criteria 평가자 사용
conciseness_evaluator = load_evaluator(
    evaluator="criteria", 
    criteria="conciseness",
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0),
    )

# 2번째 샘플에 대해 평가 수행
conciseness_result = conciseness_evaluator.evaluate_strings(
    input=question,                   # 질문 
    prediction=groq_prediction,       # 평가 대상: LLM 모델의 예측
)

# 결과 출력
conciseness_result

In [ ]:
print(conciseness_result['reasoning'])

In [ ]:
# criteria 직접 지정
criteria_evaluator = load_evaluator(
    evaluator="criteria", 
    criteria={
        "relevance": "Does the answer appropriately address the question?",
        "conciseness": "Does the answer convey the key information without unnecessary details?",
        },
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0),
    )

# 2번째 샘플에 대해 평가 수행
criteria_result = criteria_evaluator.evaluate_strings(
    input=question,              # 질문 
    prediction=groq_prediction,      # 평가 대상: LLM 모델의 예측
)

# 결과 출력
criteria_result

In [ ]:
print(criteria_result['reasoning'])

In [ ]:
# score_string 평가자 사용
score_string_evaluator = load_evaluator(
    evaluator="score_string", 
    criteria={
        "relevance": "How relevant is the answer to the question on a scale of 1-10?",
    },
    normalize_by=10,
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0),
    )

# 2번째 샘플에 대해 평가 수행
score_string_result = score_string_evaluator.evaluate_strings(
    input=question,                     # 질문 
    prediction=groq_prediction,         # 평가 대상: LLM 모델의 예측
)

# 결과 출력
score_string_result


In [ ]:
print(score_string_result['reasoning'])

`(3) Criteria Evaluation (With lables) - 참조 레이블(ground truth)이 주어진 상황에서 모델 출력의 품질을 평가`

1. "labeled_criteria" 평가기:
   - 목적: 참조 레이블을 고려하여 예측이 주어진 기준을 만족하는지 평가
   - 출력: 이진 점수 (예: Yes/No 또는 1/0)

2. "labeled_score_string" 평가기:
   - 목적: 참조 레이블과 비교하여 예측의 품질을 수치로 평가
   - 출력: 수치 점수 (기본적으로 1-10 척도)


In [ ]:
from langchain.evaluation import load_evaluator

# labeled_criteria 평가자 사용
labeled_crieria_evaluator = load_evaluator(
    evaluator="labeled_criteria", 
    criteria="correctness",
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0),
    )

# 2번째 샘플에 대해 평가 수행
labeled_crieria_eval_result = labeled_crieria_evaluator.evaluate_strings(
    input=question,               # 평가에 고려할 내용: 질문
    prediction=groq_prediction,   # 평가 대상: LLM 모델의 예측
    reference=ground_truth,       # 평가 기준: 정답
)

# 결과 출력
labeled_crieria_eval_result

In [ ]:
print(labeled_crieria_eval_result['reasoning'])

In [ ]:
# labeled_score_string 평가자 사용
labeled_score_string_evaluator = load_evaluator(
    evaluator="labeled_score_string", 
    criteria="correctness",
    normalize_by=10,
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0),
    )

# 2번째 샘플에 대해 평가 수행
labeled_score_string_eval_result = labeled_score_string_evaluator.evaluate_strings(
    input=question,               # 평가에 고려할 내용: 질문
    prediction=groq_prediction,   # 평가 대상: LLM 모델의 예측
    reference=ground_truth,       # 평가 기준: 정답
)

# 결과 출력
labeled_score_string_eval_result

In [ ]:
print(labeled_score_string_eval_result['reasoning'])

In [ ]:
# 프롬프트를 직접 작성하여 평가 수행 (한국어로 평가)

from langchain.evaluation import load_evaluator, EvaluatorType
from langchain_openai import ChatOpenAI


from langchain_core.prompts import PromptTemplate

correctness_eval_template = """Respond Y or N based on how accurate the given response is compared to the expected answer. Grade only based on the rubric and expected answer:

Grading Rubric: {criteria}
Question: {input}
Expected Answer: {reference}

DATA:
---------
Actual Response: {output}
---------
Write out your explanation for how accurate the response is compared to the expected answer, considering factual correctness and completeness. Then respond with Y or N on a new line. (in Korean)"""

correctness_eval_prompt = PromptTemplate.from_template(correctness_eval_template)

# Correctness: 생성된 답변이 ground truth와 비교하여 정확하고 사실적인지 평가
correctness_evaluator = load_evaluator(
    evaluator="labeled_criteria", 
    criteria="correctness",  
    llm=ChatOpenAI(model="gpt-4o", temperature=0.0), 
    prompt=correctness_eval_prompt # 사용자 지정 프롬프트 사용
    )


# 2번째 샘플에 대해 평가 수행
correctness_eval_result = correctness_evaluator.evaluate_strings(
    input=question,
    prediction=groq_prediction,
    reference=ground_truth,
)

print(f'Correctness Score: {correctness_eval_result["score"]}')
print(f'Correctness Reasoniong: {correctness_eval_result["reasoning"]}')


In [ ]:
correctness_eval_result

In [ ]:
# labled_score_string 평가
correctness_score_template = """
[Instruction]
Please act as an impartial judge and evaluate the correctness of the AI assistant's response compared to the ground truth. 
{criteria}

[Ground Truth]
{reference}

Begin your evaluation by providing a short explanation. Be as objective as possible. Answer in Korean. After providing your explanation, you must rate the response on a scale of 1 to 5 by strictly following this format: "[[rating]]", for example: "Rating: [[3]]".

[Question]
{input}

[The Start of Assistant's Answer]
{prediction}
[The End of Assistant's Answer]
"""

correctness_score_prompt = PromptTemplate.from_template(correctness_score_template)

correctness_criteria = {
    "correctness": """
Score 1: The answer is completely incorrect or contradicts the ground truth.
Score 2: The answer contains major factual errors or misunderstandings of the ground truth.
Score 3: The answer is partially correct but contains some factual errors or omissions.
Score 4: The answer is mostly correct with only minor inaccuracies or omissions.
Score 5: The answer is completely correct and fully aligned with the ground truth."""
}

correctness_score_evaluator = load_evaluator(
    evaluator="labeled_score_string",  
    criteria=correctness_criteria,   
    normalize_by=5,                
    llm=ChatOpenAI(model="gpt-4o", temperature=0.0), 
    prompt=correctness_score_prompt 
)

# 2번째 샘플에 대해 평가 수행
correctness_score_result = correctness_score_evaluator.evaluate_strings(
    input=question,                   # 평가에 고려할 내용: 질문
    prediction=groq_prediction,       # 평가 대상: LLM 모델의 예측
    reference=ground_truth,           # 평가 기준: 정답
)

print(f'Correctness Score: {correctness_score_result["score"]}')
print(f'Correctness Reasoning: {correctness_score_result["reasoning"]}')